<a href="https://colab.research.google.com/github/hfnvbh/colab-trt-pose/blob/master/trt-pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pose detection demo with trt-pose
This notebook  based on [NVIDIA-AI-IOT/trt_pose](https://github.com/NVIDIA-AI-IOT/trt_pose) project and uses its [live demo](https://github.com/NVIDIA-AI-IOT/trt_pose/blob/master/tasks/human_pose/live_demo.ipynb) adapted to work with youtube videos in Google Colab.


##Install TensorRT
Since Nvidia give access only for registered users, please visit [TensorRT home page](https://developer.nvidia.com/tensorrt) and download .deb package by yourself. Upload it to some cloud drive and paste link to .deb package into next cell. Also check installation instructions at TensorRT download page, fix filenames used in dpkg and apt-key command if needed.

Please notice that this notebook using specific version of TensorRT. It's needed TensorRT 5 .deb package for Cuda 10.1 and Ubuntu 18.04.

In [0]:
tensorRTlink="https://www.dropbox.com/s/p0sfni1dzb359hk/nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb"

In [0]:
import os
os.system('wget %s' %tensorRTlink)

!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.1-trt5.1.5.0-ga-20190427_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.1-trt5.1.5.0-ga-20190427/7fa2af80.pub

Install specific versions of packages to avoid dependency issues with and then install TensorRT

In [0]:
!apt-get install -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.1 libnvinfer-dev=5.1.5-1+cuda10.1
!apt-get install -y --no-install-recommends python3-libnvinfer=5.1.5-1+cuda10.1 python3-libnvinfer-dev=5.1.5-1+cuda10.1
!apt-get update
!apt-get install tensorrt

Check installation results.
Expexted output (for TensorRT 5.1.5):

||||||
|-|-|-|-|-|
| ii | libnvinfer-dev | 5.1.5-1+cuda10.1 | amd64 | TensorRT development libraries and headers |
| ii | libnvinfer-samples | 5.1.5-1+cuda10.1 | all | TensorRT samples and documentation |
| ii | libnvinfer5 | 5.1.5-1+cuda10.1 | amd64 | TensorRT runtime libraries |
| ii | python3-libnvinfer | 5.1.5-1+cuda10.1 | amd64 | Python 3 bindings for TensorRT |
| ii | python3-libnvinfer-dev | 5.1.5-1+cuda10.1 | amd64 | Python 3 development package for TensorRT |
| ii | tensorrt | 5.1.5.0-1+cuda10.1 | amd64 | Meta package of TensorRT |

In [0]:
!dpkg -l | grep TensorRT

## Clone needed repos
You need [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt), [trt_pose](https://github.com/NVIDIA-AI-IOT/trt_pose) itself and scripts from [colab-trt-pose](https://github.com/hfnvbh/colab-trt-pose)

In [0]:
!apt-get install libprotobuf* protobuf-compiler ninja-build
!git clone https://github.com/NVIDIA-AI-IOT/torch2trt
!cd torch2trt && python setup.py install --plugins

In [0]:
!pip3 install tqdm cython pycocotools
!apt-get install python3-matplotlib
!git clone https://github.com/NVIDIA-AI-IOT/trt_pose
!cd trt_pose && python3 setup.py install

In [0]:
!git clone https://github.com/hfnvbh/colab-trt-pose

Also download weigths for [resnet](https://drive.google.com/open?id=1XYDdCUdiF2xxx4rznmLb62SdOUZuoNbd) used in trt-pose.

In [0]:
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XYDdCUdiF2xxx4rznmLb62SdOUZuoNbd' -O 'resnet18_baseline_att_224x224_A_epoch_249.pth'

Copy JSON file which describes the human pose task to working directory.

In [0]:
!cp trt_pose/tasks/human_pose/human_pose.json .

Convert model to trt.

In [0]:
!python colab-trt-pose/convert_model_to_trt.py

## Install packages for work with youtube videos

In [0]:
!pip install -q youtube-dl visdom
from IPython.display import YouTubeVideo

##Run example 

In [0]:
YOUTUBE_ID = 'RXABo9hm8B8'
YouTubeVideo(YOUTUBE_ID)

In [0]:
!rm -df youtube.mp4
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

Cut and run demo only on first 5 seconds.

In [0]:
!ffmpeg -y -loglevel error -i youtube.mp4 -t 5 video.mp4

Run script with processing video.

In [0]:
!python colab-trt-pose/process_video_trt_pose.py video.mp4

Output video has .avi fromat, so convert it back to .mp4

In [0]:
!ffmpeg -y -loglevel error -i output.avi output.mp4 

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4')